# Measure regionprops plugin

Create a plugin to profile morphological features from cells. The plugin should enable users to:

1) Read an image stack
2) Segment cells
3) Measure image features of each cell
4) Store the result as a CSV file 

In [57]:
from skimage import data
from skimage.util import img_as_float
import matplotlib.pyplot as plt
import numpy as np

import napari
from magicgui import magicgui

from napari.layers import Image
from napari.types import LabelsData
import pathlib

In [58]:
IMAGE2D = data.cells3d()[30,1]

In [59]:
def segment_cells(img):
    from cellpose import models
    
    model = models.CellposeModel(gpu=True)
    img = img_as_float(img)
    mask, __, __ = model.eval(img)
    
    return mask

def get_properties(mask, img):
    from skimage.measure import regionprops 
    
    return regionprops(mask, img)
    
def save_props_table(props, mask, properties, path):
    import pandas as pd

    prop_list = []
    for index in range(1, mask.max()):
        prop_list.append([index] + [props[index][prop_name] for prop_name in properties])
    
    prop_array = np.vstack(prop_list)
    
    df = pd.DataFrame(prop_array, columns=['CellID'] + properties)
    df.to_csv(path)

In [60]:
@magicgui(path={'label': 'Path to save',
                'widget_type': 'FileEdit',
                'mode': 'w',
                'filter': '*.csv'})
def measure_regionprops(
    img: 'napari.layers.Image', 
    area: 'bool',
    eccentricity: 'bool',
    perimeter: 'bool',
    intensity_mean: 'bool',
    path: 'pathlib.Path.home()'):

    properties = []

    if area:
        properties.append('area')
    if eccentricity:
        properties.append('eccentricity')
    if perimeter:
        properties.append('perimeter')
    if intensity_mean:
        properties.append('intensity_mean')
    
    mask = segment_cells(img.data)
    props = get_properties(mask, img.data)
    save_props_table(props, mask, properties, path)
    

viewer = napari.view_image(IMAGE2D)
viewer.window.add_dock_widget(measure_regionprops)